ToDo:
    - find out how "condition" relates to group membership

In [1]:
import mne
import pandas as pd
import numpy as np
import os

In [2]:
mne.sys_info()

Platform:      Linux-5.11.0-37-generic-x86_64-with-debian-bullseye-sid
Python:        3.7.6 (default, Jan  8 2020, 19:59:22)  [GCC 7.3.0]
Executable:    /home/michael/anaconda3/bin/python
CPU:           x86_64: 16 cores
Memory:        15.0 GB

mne:           0.23.4
numpy:         1.18.1 {blas=mkl_rt, lapack=mkl_rt}
scipy:         1.4.1
matplotlib:    3.1.3 {backend=module://ipykernel.pylab.backend_inline}

sklearn:       0.22.1
numba:         0.48.0
nibabel:       Not found
nilearn:       Not found
dipy:          Not found
cupy:          Not found
pandas:        1.0.1
mayavi:        Not found
pyvista:       Not found
vtk:           Not found


In [3]:
rootdir = '/home/michael/data/derivatives/reaction_epochs'

In [4]:
bdi = pd.read_csv('/home/michael/git/master_thesis/data/bdi_ci.tsv', sep='\t',
                  dtype={'subject': str})
des = pd.read_csv('/home/michael/git/master_thesis/data/des_ci.tsv', sep='\t',
                  dtype={'subject': str})
sps = pd.read_csv('/home/michael/git/master_thesis/data/sps_ci.tsv', sep='\t',
                  dtype={'subject': str})

In [5]:
bdi

,bdi_sum_score,bdi_std,upper_ci,lower_ci,subject
0,8,0.669043,12.095281,3.904719,02
1,3,0.358569,7.095281,-1.095281,03
2,0,0.000000,4.095281,-4.095281,04
3,0,0.000000,4.095281,-4.095281,05
4,8,0.669043,12.095281,3.904719,06
5,7,0.577350,11.095281,2.904719,07
6,6,0.560612,10.095281,1.904719,08
7,6,0.643650,10.095281,1.904719,09
8,7,0.483046,11.095281,2.904719,10
9,4,0.402374,8.095281,-0.095281,11


subj_id = ['02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18',
           '19', '20', '21', '22', '23', '24', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37',
           '38', '39']

bdi['subject'] = subj_id
des['subject'] = subj_id
sps['subject'] = subj_id

In [6]:
counter = 0
directory = '/home/michael/data/derivatives/results/erp'

if not os.path.exists(directory):
    os.makedirs(directory)

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".fif"):
            print (filepath)
            print((filepath.split('/')[-2].split('-')[-1]))
            subject = (filepath.split('/')[-2].split('-')[-1])[-2:]
             # get epochs
            cue_epochs = mne.read_epochs(filepath, preload=True)


            correct_epochs = cue_epochs['correct_incongruent', 'correct_congruent']

            correct_epochs.apply_baseline((-0.300, -0.050))
            correct_epochs_test = correct_epochs['condition == 0']
            correct_epochs_baseline = correct_epochs['condition == 1']
            correct_epochs_2 = correct_epochs['condition == 2']
            correct_epochs_3 = correct_epochs['condition == 3']

            # epochs to pandas dataframe
            df_correct_baseline = correct_epochs_baseline.to_data_frame(long_format=True)
            df_correct_baseline['exp_condition'] = 'baseline'
            df_correct_2 = correct_epochs_2.to_data_frame(long_format=True)
            df_correct_2['exp_condition'] = '2'
            df_correct_3 = correct_epochs_3.to_data_frame(long_format=True)
            df_correct_3['exp_condition'] = '3'
            df_correct = df_correct_baseline.append(df_correct_2)
            df_correct = df_correct.append(df_correct_3)
            #display(correct_epochs_baseline.metadata)
            if correct_epochs_baseline.metadata['negative_first'].iloc[0] == True:
                group = 'negative_first'
            elif correct_epochs_baseline.metadata['negative_first'].iloc[0] == False:
                group = 'positive_first'
            df_correct['group'] = group
            df_correct['bdi_sum_score'] = bdi.loc[bdi['subject'] == subject[-2:]]['bdi_sum_score'].values[0]
            df_correct['des_sum_score'] = des.loc[des['subject'] == subject[-2:]]['des_sum_score'].values[0]
            df_correct['sps_sum_score'] = sps.loc[des['subject'] == subject[-2:]]['sp_sum_score'].values[0]

            #display(df_correct)
            
            if counter == 0:
                CRN = df_correct[((df_correct["time"] >= 0) & (df_correct["time"] <= 100))
                           & ((df_correct["channel"] == 'Fz') |
                              (df_correct["channel"] == 'FCz') |
                              (df_correct["channel"] == 'Cz') | 
                              (df_correct["channel"] == 'CPz') |
                              (df_correct["channel"] == 'Pz'))]
                CRN = CRN.assign(subject=subject)
                CRN.to_csv('/home/michael/data/derivatives/results/erp/crn_sub-' + subject + '.tsv', sep='\t', index=False)

                
            else:
                CRN_temp = df_correct[((df_correct["time"] >= 0) & (df_correct["time"] <= 100))
                           & ((df_correct["channel"] == 'Fz') |
                              (df_correct["channel"] == 'FCz') |
                              (df_correct["channel"] == 'Cz') | 
                              (df_correct["channel"] == 'CPz') |
                              (df_correct["channel"] == 'Pz'))]
                CRN_temp = CRN_temp.assign(subject=subject)
                CRN_temp.to_csv('/home/michael/data/derivatives/results/erp/crn_sub-' + subject + '.tsv', sep='\t', index=False)

                CRN = CRN.append(CRN_temp)
                display(CRN_temp)
                display(CRN)
            counter +=1


/home/michael/data/derivatives/reaction_epochs/sub-007/sub-007-reaction_epochs-epo.fif
007
Reading /home/michael/data/derivatives/reaction_epochs/sub-007/sub-007-reaction_epochs-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 64) active
    Found the data of interest:
        t =   -1500.00 ...    1500.00 ms
        0 CTF compensation matrices available
Adding metadata with 9 columns
Replacing existing metadata with 9 columns
1229 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Applying baseline correction (mode: mean)
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "condition" to "category"...
Convertin

,condition,epoch,time,channel,ch_type,value,exp_condition,group,bdi_sum_score,des_sum_score,sps_sum_score,subject
12318,correct_incongruent,48,0,Pz,eeg,-2.944312,baseline,negative_first,6,70,6,09
12319,correct_incongruent,48,0,CPz,eeg,-6.759031,baseline,negative_first,6,70,6,09
12325,correct_incongruent,48,0,Fz,eeg,10.390014,baseline,negative_first,6,70,6,09
12334,correct_incongruent,48,0,FCz,eeg,8.802264,baseline,negative_first,6,70,6,09
12335,correct_incongruent,48,0,Cz,eeg,0.706449,baseline,negative_first,6,70,6,09
...,...,...,...,...,...,...,...,...,...,...,...,...
9573406,correct_incongruent,842,94,Pz,eeg,-0.266721,3,negative_first,6,70,6,09
9573407,correct_incongruent,842,94,CPz,eeg,1.114624,3,negative_first,6,70,6,09
9573413,correct_incongruent,842,94,Fz,eeg,-4.411711,3,negative_first,6,70,6,09
9573422,correct_incongruent,842,94,FCz,eeg,-1.289859,3,negative_first,6,70,6,09


,condition,epoch,time,channel,ch_type,value,exp_condition,group,bdi_sum_score,des_sum_score,sps_sum_score,subject
12318,correct_congruent,50,0,Pz,eeg,6.147827,baseline,negative_first,7,62,4,07
12319,correct_congruent,50,0,CPz,eeg,7.788848,baseline,negative_first,7,62,4,07
12325,correct_congruent,50,0,Fz,eeg,3.760848,baseline,negative_first,7,62,4,07
12334,correct_congruent,50,0,FCz,eeg,4.693713,baseline,negative_first,7,62,4,07
12335,correct_congruent,50,0,Cz,eeg,4.384005,baseline,negative_first,7,62,4,07
...,...,...,...,...,...,...,...,...,...,...,...,...
9573406,correct_incongruent,842,94,Pz,eeg,-0.266721,3,negative_first,6,70,6,09
9573407,correct_incongruent,842,94,CPz,eeg,1.114624,3,negative_first,6,70,6,09
9573413,correct_incongruent,842,94,Fz,eeg,-4.411711,3,negative_first,6,70,6,09
9573422,correct_incongruent,842,94,FCz,eeg,-1.289859,3,negative_first,6,70,6,09


/home/michael/data/derivatives/reaction_epochs/sub-018/sub-018-reaction_epochs-epo.fif
018
Reading /home/michael/data/derivatives/reaction_epochs/sub-018/sub-018-reaction_epochs-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 64) active
    Found the data of interest:
        t =   -1500.00 ...    1500.00 ms
        0 CTF compensation matrices available
Adding metadata with 9 columns
Replacing existing metadata with 9 columns
1223 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Applying baseline correction (mode: mean)
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "condition" to "category"...
Convertin

,condition,epoch,time,channel,ch_type,value,exp_condition,group,bdi_sum_score,des_sum_score,sps_sum_score,subject
12318,correct_congruent,50,0,Pz,eeg,-1.016117,baseline,negative_first,0,63,3,18
12319,correct_congruent,50,0,CPz,eeg,-2.172129,baseline,negative_first,0,63,3,18
12325,correct_congruent,50,0,Fz,eeg,-4.301439,baseline,negative_first,0,63,3,18
12334,correct_congruent,50,0,FCz,eeg,1.579124,baseline,negative_first,0,63,3,18
12335,correct_congruent,50,0,Cz,eeg,-1.279637,baseline,negative_first,0,63,3,18
...,...,...,...,...,...,...,...,...,...,...,...,...
9327006,correct_congruent,845,94,Pz,eeg,7.924622,3,negative_first,0,63,3,18
9327007,correct_congruent,845,94,CPz,eeg,5.004389,3,negative_first,0,63,3,18
9327013,correct_congruent,845,94,Fz,eeg,-2.799053,3,negative_first,0,63,3,18
9327022,correct_congruent,845,94,FCz,eeg,6.953091,3,negative_first,0,63,3,18


...

,condition,epoch,time,channel,ch_type,value,exp_condition,group,bdi_sum_score,des_sum_score,sps_sum_score,subject
12318,correct_incongruent,50,0,Pz,eeg,-2.235261,baseline,negative_first,5,72,22,28
12319,correct_incongruent,50,0,CPz,eeg,-0.220324,baseline,negative_first,5,72,22,28
12325,correct_incongruent,50,0,Fz,eeg,-3.143450,baseline,negative_first,5,72,22,28
12334,correct_incongruent,50,0,FCz,eeg,8.055111,baseline,negative_first,5,72,22,28
12335,correct_incongruent,50,0,Cz,eeg,11.968459,baseline,negative_first,5,72,22,28
...,...,...,...,...,...,...,...,...,...,...,...,...
9425566,correct_incongruent,845,94,Pz,eeg,5.263699,3,negative_first,5,72,22,28
9425567,correct_incongruent,845,94,CPz,eeg,6.102737,3,negative_first,5,72,22,28
9425573,correct_incongruent,845,94,Fz,eeg,2.403199,3,negative_first,5,72,22,28
9425582,correct_incongruent,845,94,FCz,eeg,4.010714,3,negative_first,5,72,22,28


,condition,epoch,time,channel,ch_type,value,exp_condition,group,bdi_sum_score,des_sum_score,sps_sum_score,subject
12318,correct_congruent,50,0,Pz,eeg,6.147827,baseline,negative_first,7,62,4,07
12319,correct_congruent,50,0,CPz,eeg,7.788848,baseline,negative_first,7,62,4,07
12325,correct_congruent,50,0,Fz,eeg,3.760848,baseline,negative_first,7,62,4,07
12334,correct_congruent,50,0,FCz,eeg,4.693713,baseline,negative_first,7,62,4,07
12335,correct_congruent,50,0,Cz,eeg,4.384005,baseline,negative_first,7,62,4,07
...,...,...,...,...,...,...,...,...,...,...,...,...
9425566,correct_incongruent,845,94,Pz,eeg,5.263699,3,negative_first,5,72,22,28
9425567,correct_incongruent,845,94,CPz,eeg,6.102737,3,negative_first,5,72,22,28
9425573,correct_incongruent,845,94,Fz,eeg,2.403199,3,negative_first,5,72,22,28
9425582,correct_incongruent,845,94,FCz,eeg,4.010714,3,negative_first,5,72,22,28


/home/michael/data/derivatives/reaction_epochs/sub-013/sub-013-reaction_epochs-epo.fif
013
Reading /home/michael/data/derivatives/reaction_epochs/sub-013/sub-013-reaction_epochs-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 64) active
    Found the data of interest:
        t =   -1500.00 ...    1500.00 ms
        0 CTF compensation matrices available
Adding metadata with 9 columns
Replacing existing metadata with 9 columns
1223 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Applying baseline correction (mode: mean)
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "condition" to "category"...
Convertin

,condition,epoch,time,channel,ch_type,value,exp_condition,group,bdi_sum_score,des_sum_score,sps_sum_score,subject
12318,correct_incongruent,49,0,Pz,eeg,0.745618,baseline,positive_first,1,64,9,13
12319,correct_incongruent,49,0,CPz,eeg,5.219995,baseline,positive_first,1,64,9,13
12325,correct_incongruent,49,0,Fz,eeg,13.169006,baseline,positive_first,1,64,9,13
12334,correct_incongruent,49,0,FCz,eeg,19.612789,baseline,positive_first,1,64,9,13
12335,correct_incongruent,49,0,Cz,eeg,9.886598,baseline,positive_first,1,64,9,13
...,...,...,...,...,...,...,...,...,...,...,...,...
9253086,correct_congruent,1245,94,Pz,eeg,20.169521,3,positive_first,1,64,9,13
9253087,correct_congruent,1245,94,CPz,eeg,12.899058,3,positive_first,1,64,9,13
9253093,correct_congruent,1245,94,Fz,eeg,-32.446943,3,positive_first,1,64,9,13
9253102,correct_congruent,1245,94,FCz,eeg,-31.186029,3,positive_first,1,64,9,13


,condition,epoch,time,channel,ch_type,value,exp_condition,group,bdi_sum_score,des_sum_score,sps_sum_score,subject
12318,correct_congruent,50,0,Pz,eeg,6.147827,baseline,negative_first,7,62,4,07
12319,correct_congruent,50,0,CPz,eeg,7.788848,baseline,negative_first,7,62,4,07
12325,correct_congruent,50,0,Fz,eeg,3.760848,baseline,negative_first,7,62,4,07
12334,correct_congruent,50,0,FCz,eeg,4.693713,baseline,negative_first,7,62,4,07
12335,correct_congruent,50,0,Cz,eeg,4.384005,baseline,negative_first,7,62,4,07
...,...,...,...,...,...,...,...,...,...,...,...,...
9253086,correct_congruent,1245,94,Pz,eeg,20.169521,3,positive_first,1,64,9,13
9253087,correct_congruent,1245,94,CPz,eeg,12.899058,3,positive_first,1,64,9,13
9253093,correct_congruent,1245,94,Fz,eeg,-32.446943,3,positive_first,1,64,9,13
9253102,correct_congruent,1245,94,FCz,eeg,-31.186029,3,positive_first,1,64,9,13


In [7]:
counter = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".fif"):
            print (filepath)
            print((filepath.split('/')[-2].split('-')[-1]))
            subject = (filepath.split('/')[-2].split('-')[-1])[-2:]
             # get epochs
            cue_epochs = mne.read_epochs(filepath, preload=True)

            incorrect_epochs = cue_epochs['incorrect_incongruent', 'incorrect_congruent']

            incorrect_epochs.apply_baseline((-0.300, -0.050))
            incorrect_epochs_test = incorrect_epochs['condition == 0']
            incorrect_epochs_baseline = incorrect_epochs['condition == 1']
            incorrect_epochs_2 = incorrect_epochs['condition == 2']
            incorrect_epochs_3 = incorrect_epochs['condition == 3']
            
            # epochs to pandas dataframe
            df_incorrect_baseline = incorrect_epochs_baseline.to_data_frame(long_format=True)
            df_incorrect_baseline['exp_condition'] = 'baseline'
            df_incorrect_2 = incorrect_epochs_2.to_data_frame(long_format=True)
            df_incorrect_2['exp_condition'] = '2'
            df_incorrect_3 = incorrect_epochs_3.to_data_frame(long_format=True)
            df_incorrect_3['exp_condition'] = '3'
            df_incorrect = df_incorrect_baseline.append(df_incorrect_2)
            df_incorrect = df_incorrect.append(df_incorrect_3)
            #display(incorrect_epochs_baseline.metadata)
            if incorrect_epochs_baseline.metadata['negative_first'].iloc[0] == True:
                group = 'negative_first'
            elif incorrect_epochs_baseline.metadata['negative_first'].iloc[0] == False:
                group = 'positive_first'
            df_incorrect['group'] = group
            df_incorrect['bdi_sum_score'] = bdi.loc[bdi['subject'] == subject[-2:]]['bdi_sum_score'].values[0]
            df_incorrect['des_sum_score'] = des.loc[des['subject'] == subject[-2:]]['des_sum_score'].values[0]
            df_incorrect['sps_sum_score'] = sps.loc[des['subject'] == subject[-2:]]['sp_sum_score'].values[0]


            #display(df_correct)
            
            if counter == 0:
                ERN = df_incorrect[((df_incorrect["time"] >= 0) & (df_incorrect["time"] <= 100))
                           & ((df_incorrect["channel"] == 'Fz') |
                              (df_incorrect["channel"] == 'FCz') |
                              (df_incorrect["channel"] == 'Cz') | 
                              (df_incorrect["channel"] == 'CPz') |
                              (df_incorrect["channel"] == 'Pz'))]
                ERN = ERN.assign(subject=subject)
                ERN.to_csv('/home/michael/data/derivatives/results/erp/ern_sub-' + subject + '.tsv', sep='\t', index=False)

                
            else:
                ERN_temp = df_incorrect[((df_incorrect["time"] >= 0) & (df_incorrect["time"] <= 100))
                           & ((df_incorrect["channel"] == 'Fz') |
                              (df_incorrect["channel"] == 'FCz') |
                              (df_incorrect["channel"] == 'Cz') | 
                              (df_incorrect["channel"] == 'CPz') |
                              (df_incorrect["channel"] == 'Pz'))]
                ERN_temp = ERN_temp.assign(subject=subject)
                ERN_temp.to_csv('/home/michael/data/derivatives/results/erp/ern_sub-' + subject + '.tsv', sep='\t', index=False)

                ERN = ERN.append(ERN_temp)
                display(ERN_temp)
                #display(ERN)
            counter +=1

/home/michael/data/derivatives/reaction_epochs/sub-007/sub-007-reaction_epochs-epo.fif
007
Reading /home/michael/data/derivatives/reaction_epochs/sub-007/sub-007-reaction_epochs-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 64) active
    Found the data of interest:
        t =   -1500.00 ...    1500.00 ms
        0 CTF compensation matrices available
Adding metadata with 9 columns
Replacing existing metadata with 9 columns
1229 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Applying baseline correction (mode: mean)
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "condition" to "category"...
Convertin

,condition,epoch,time,channel,ch_type,value,exp_condition,group,bdi_sum_score,des_sum_score,sps_sum_score,subject
12318,incorrect_incongruent,64,0,Pz,eeg,-4.986203,baseline,negative_first,6,70,6,09
12319,incorrect_incongruent,64,0,CPz,eeg,-4.791787,baseline,negative_first,6,70,6,09
12325,incorrect_incongruent,64,0,Fz,eeg,12.769535,baseline,negative_first,6,70,6,09
12334,incorrect_incongruent,64,0,FCz,eeg,7.839311,baseline,negative_first,6,70,6,09
12335,incorrect_incongruent,64,0,Cz,eeg,0.741611,baseline,negative_first,6,70,6,09
...,...,...,...,...,...,...,...,...,...,...,...,...
136286,incorrect_incongruent,841,94,Pz,eeg,6.175573,3,negative_first,6,70,6,09
136287,incorrect_incongruent,841,94,CPz,eeg,1.149558,3,negative_first,6,70,6,09
136293,incorrect_incongruent,841,94,Fz,eeg,-2.838955,3,negative_first,6,70,6,09
136302,incorrect_incongruent,841,94,FCz,eeg,0.547122,3,negative_first,6,70,6,09


/home/michael/data/derivatives/reaction_epochs/sub-018/sub-018-reaction_epochs-epo.fif
018
Reading /home/michael/data/derivatives/reaction_epochs/sub-018/sub-018-reaction_epochs-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 64) active
    Found the data of interest:
        t =   -1500.00 ...    1500.00 ms
        0 CTF compensation matrices available
Adding metadata with 9 columns
Replacing existing metadata with 9 columns
1223 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Applying baseline correction (mode: mean)
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "condition" to "category"...
Convertin

,condition,epoch,time,channel,ch_type,value,exp_condition,group,bdi_sum_score,des_sum_score,sps_sum_score,subject
12318,incorrect_congruent,118,0,Pz,eeg,-7.038443,baseline,negative_first,0,63,3,18
12319,incorrect_congruent,118,0,CPz,eeg,-4.174083,baseline,negative_first,0,63,3,18
12325,incorrect_congruent,118,0,Fz,eeg,2.458307,baseline,negative_first,0,63,3,18
12334,incorrect_congruent,118,0,FCz,eeg,2.359109,baseline,negative_first,0,63,3,18
12335,incorrect_congruent,118,0,Cz,eeg,1.103903,baseline,negative_first,0,63,3,18
...,...,...,...,...,...,...,...,...,...,...,...,...
210206,incorrect_incongruent,817,94,Pz,eeg,4.557849,3,negative_first,0,63,3,18
210207,incorrect_incongruent,817,94,CPz,eeg,8.047458,3,negative_first,0,63,3,18
210213,incorrect_incongruent,817,94,Fz,eeg,-7.995775,3,negative_first,0,63,3,18
210222,incorrect_incongruent,817,94,FCz,eeg,-10.217321,3,negative_first,0,63,3,18


/home/michael/data/derivatives/reaction_epochs/sub-011/sub-011-reaction_epochs-epo.fif
011
Reading /home/michael/data/derivatives/reaction_epochs/sub-011/sub-011-reaction_epochs-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 64) active
    Found the data of interest:
        t =   -1500.00 ...    1500.00 ms
        0 CTF compensation matrices available
Adding metadata with 9 columns
Replacing existing metadata with 9 columns
1219 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Applying baseline correction (mode: mean)
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "condition" to "category"...
Convertin

...

,condition,epoch,time,channel,ch_type,value,exp_condition,group,bdi_sum_score,des_sum_score,sps_sum_score,subject
12318,incorrect_incongruent,50,0,Pz,eeg,15.497354,baseline,negative_first,2,62,4,16
12319,incorrect_incongruent,50,0,CPz,eeg,-2.955236,baseline,negative_first,2,62,4,16
12325,incorrect_incongruent,50,0,Fz,eeg,-8.239772,baseline,negative_first,2,62,4,16
12334,incorrect_incongruent,50,0,FCz,eeg,-2.958629,baseline,negative_first,2,62,4,16
12335,incorrect_incongruent,50,0,Cz,eeg,6.557505,baseline,negative_first,2,62,4,16
...,...,...,...,...,...,...,...,...,...,...,...,...
382686,incorrect_incongruent,822,94,Pz,eeg,-7.343913,3,negative_first,2,62,4,16
382687,incorrect_incongruent,822,94,CPz,eeg,-5.016189,3,negative_first,2,62,4,16
382693,incorrect_incongruent,822,94,Fz,eeg,-3.606177,3,negative_first,2,62,4,16
382702,incorrect_incongruent,822,94,FCz,eeg,-0.604039,3,negative_first,2,62,4,16


/home/michael/data/derivatives/reaction_epochs/sub-028/sub-028-reaction_epochs-epo.fif
028
Reading /home/michael/data/derivatives/reaction_epochs/sub-028/sub-028-reaction_epochs-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 64) active
    Found the data of interest:
        t =   -1500.00 ...    1500.00 ms
        0 CTF compensation matrices available
Adding metadata with 9 columns
Replacing existing metadata with 9 columns
1214 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Applying baseline correction (mode: mean)
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "condition" to "category"...
Convertin

,condition,epoch,time,channel,ch_type,value,exp_condition,group,bdi_sum_score,des_sum_score,sps_sum_score,subject
12318,incorrect_incongruent,135,0,Pz,eeg,-1.767037,baseline,negative_first,5,72,22,28
12319,incorrect_incongruent,135,0,CPz,eeg,4.097989,baseline,negative_first,5,72,22,28
12325,incorrect_incongruent,135,0,Fz,eeg,6.919587,baseline,negative_first,5,72,22,28
12334,incorrect_incongruent,135,0,FCz,eeg,8.927175,baseline,negative_first,5,72,22,28
12335,incorrect_incongruent,135,0,Cz,eeg,7.219892,baseline,negative_first,5,72,22,28
...,...,...,...,...,...,...,...,...,...,...,...,...
284126,incorrect_incongruent,825,94,Pz,eeg,13.116459,3,negative_first,5,72,22,28
284127,incorrect_incongruent,825,94,CPz,eeg,13.212569,3,negative_first,5,72,22,28
284133,incorrect_incongruent,825,94,Fz,eeg,-8.197358,3,negative_first,5,72,22,28
284142,incorrect_incongruent,825,94,FCz,eeg,-0.869905,3,negative_first,5,72,22,28


/home/michael/data/derivatives/reaction_epochs/sub-013/sub-013-reaction_epochs-epo.fif
013
Reading /home/michael/data/derivatives/reaction_epochs/sub-013/sub-013-reaction_epochs-epo.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 64) active
    Found the data of interest:
        t =   -1500.00 ...    1500.00 ms
        0 CTF compensation matrices available
Adding metadata with 9 columns
Replacing existing metadata with 9 columns
1223 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Applying baseline correction (mode: mean)
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...
Converting "condition" to "category"...
Convertin

,condition,epoch,time,channel,ch_type,value,exp_condition,group,bdi_sum_score,des_sum_score,sps_sum_score,subject
12318,incorrect_incongruent,66,0,Pz,eeg,15.547927,baseline,positive_first,1,64,9,13
12319,incorrect_incongruent,66,0,CPz,eeg,11.588999,baseline,positive_first,1,64,9,13
12325,incorrect_incongruent,66,0,Fz,eeg,-0.078671,baseline,positive_first,1,64,9,13
12334,incorrect_incongruent,66,0,FCz,eeg,7.510279,baseline,positive_first,1,64,9,13
12335,incorrect_incongruent,66,0,Cz,eeg,8.339175,baseline,positive_first,1,64,9,13
...,...,...,...,...,...,...,...,...,...,...,...,...
530526,incorrect_incongruent,1225,94,Pz,eeg,27.512071,3,positive_first,1,64,9,13
530527,incorrect_incongruent,1225,94,CPz,eeg,13.622687,3,positive_first,1,64,9,13
530533,incorrect_incongruent,1225,94,Fz,eeg,-27.077373,3,positive_first,1,64,9,13
530542,incorrect_incongruent,1225,94,FCz,eeg,-15.410481,3,positive_first,1,64,9,13


In [8]:
CRN.to_csv('/home/michael/data/derivatives/results/erp/crns.tsv', sep='\t', index=False)
ERN.to_csv('/home/michael/data/derivatives/results/erp/erns.tsv', sep='\t', index=False)